# Getting started

## Installation

See the [README](https://github.com/ibis-project/ibis-tutorial#setup) for up-to-date installation instructions!

## Download some data

There are other ways to get example data, but we'll start by downloading the penguins dataset.
$^1$.

In [ ]:
from pathlib import Path

import duckdb
from packaging.version import parse as vparse

duck_version = vparse(duckdb.__version__)

ddb_file = Path("palmer_penguins.ddb")


if not ddb_file.exists():
    import urllib.request

    urllib.request.urlretrieve(
        f"https://storage.googleapis.com/ibis-tutorial-data/penguins/0.{duck_version.minor}/palmer_penguins.ddb",
        ddb_file,
    )

DuckDB is similar to sqlite -- we have a single file on disk (or an in-memory
connection) that we can operate on.

## Intro

We can begin by importing Ibis and firing up a connection to DuckDB! (DuckDB is
fast and runs locally, so it's handy for lots of use-cases, including
tutorials).

In [ ]:
import ibis

con = ibis.duckdb.connect("palmer_penguins.ddb", read_only=True)

**Note**: when you connect to a DuckDB database file, DuckDB will create a
  lock-file to prevent data corruption.  If you see a `palmer_penguins.ddb.wal`
  file, you can safely ignore it. It will get cleaned up automatically.


Now we have a connection, we can start by looking around.  Are there any tables
in this database (one would hope)?


In [ ]:
con.list_tables()

Yep, there's a table called `penguins`. Let's take a look at it.

In [ ]:
penguins = con.table("penguins")

By default, you'll get a `repr` showing the schema of the table, the column
names and the dtype of each column.

In [ ]:
penguins

If we run `head` to peek at the data, you'll notice that we don't actually see
data (yet), what's going on?

In [ ]:
penguins.head()

Ibis has a deferred execution model.  It builds up expressions based on what you
ask it to do, and then executes those expressions on request.

In this case, our query isn't too involved, we want to see the first few rows of
the `penguins` table.

We can do that by asking for the results of this query as a `pandas.DataFrame`:

In [ ]:
penguins.head().to_pandas()

Or a `pyarrow.Table`:

In [ ]:
penguins.head().to_pyarrow()

Or a `polars.DataFrame`:

In [ ]:
penguins.head().to_polars()

We'll get into more detail about what Ibis is doing a bit later on.  For now,
the important point is that Ibis is deferred.

## Interactive Mode

Remember when we said Ibis is deferred? Sometimes you want eager execution so
you can explore a dataset.  For most of this tutorial, we'll turn on
`interactive` mode, where Ibis will eagerly execute as much of the query as it
needs to in order to show you the first 10 rows of the result.

In [ ]:
ibis.options.interactive = True

In interactive mode, we use `rich` to render the output inline:

In [ ]:
penguins.head()

## Tables and Columns

`penguins` is a table!  A table is a collection of one or more columns, each with a specific datatype.

In [ ]:
penguins.head()

In [ ]:
type(penguins)

We can look at a single column of that table using the column name as an attribute:

In [ ]:
penguins.species

What kind of column is `species`? It's a `StringColumn`!

In [ ]:
type(penguins.species)

## Ibis "verbs", or, stuff you can do to a table

The rest of this notebook covers some of the general methods you can use to
alter the output of a particular table.

We'll cover, in order, `filter`, `select`, `drop`, `mutate`, `order_by`,
`aggregate`, and `group_by`. Time to dive in!

## Filter

A filter allows you to view a subset of the rows in a table, based on some condition.

For instance, we might want to only view penguin data for the Adelie species:

In [ ]:
penguins.filter(penguins.species == "Adelie")

You can also combine multiple filters, across multiple columns.

We can subset the data down to Adelie penguins residing on the island of Dream:

In [ ]:
expr = penguins.filter((penguins.island == "Dream") & (penguins.species == "Adelie"))
expr

Above we combined two filters using `&`, you can also pass them in as individual arguments:

In [ ]:
expr = penguins.filter(
        penguins.island == "Dream",
        penguins.species == "Adelie",
)
expr

In [ ]:
ibis.to_sql(expr)

## Select

Filter filters, Select selects (there's a pattern here).
If you only want a subset of the columns in the original table, you can select
those columns explicitly.

You can refer to the columns using strings:

In [ ]:
penguins.select("species", "island", "year")

Or you can use explicit references to the `Column` objects:

In [ ]:
penguins.select(penguins.species, penguins.island, penguins.year)

Or you can mix and match:

In [ ]:
penguins.select("species", "island", penguins.year)

## Drop

Drop is nearly the same as Select, but rather than explicitly choosing the
columns to display, we explicitly choose the columns to _not_ display.

And as with `select`, you can specify the columns as strings:

In [ ]:
penguins.drop("sex", "year")

Or you can use explicit references to the `Column` objects:

In [ ]:
penguins.drop(penguins.sex, penguins.year)

Or you can mix and match:

In [ ]:
penguins.drop("sex", penguins.year)

## Mutate

Everything we've seen so far has been subtractive -- removing rows or columns.
What about _adding_ columns?

That's what `mutate` is for! You can create a new column -- either as a
function of other existing columns (for example, converting units):

In [ ]:
penguins.mutate(bill_length_cm=penguins.bill_length_mm / 10)

Or you can create a new column and populate it with some literal value:

In [ ]:
penguins.mutate(my_favorite_number=ibis.literal(41))

## On immutability

We've filtered, selected, dropped, and mutated this `penguins` table quite a bit.

In [ ]:
penguins

And yet, notice that none of our changes persist -- the base table for our query isn't altered.
The query (or expression) is a recipe of things to do with the base table (`penguins`).

If you want to keep an expression around, you can assign it to a variable:

In [ ]:
expr = penguins.select("species", "island")
expr

**Note**: Every time you execute an expression (via interactive mode, or
  `to_pandas`, or similar), the entire expression gets executed, starting from
  the base table.  DuckDB is very fast and this dataset is very small, so the
  delay is unnoticeable, but for very large datasets, it might become more
  pronounced.  There is functionality to `cache` intermediate results that isn't
  covered in this tutorial, but you can read more about it in [the
  docs](https://ibis-project.org/reference/expression-tables.html#ibis.expr.types.relations.Table.cache).

## Method chaining

You can build up complicated queries by chaining together Ibis methods. The
output of many Ibis methods is a table (just like `penguins`!) and we can
continue calling table methods until we're satisfied.  Or until we end up with
something that _isn't_ a table.  More on that later.

In [ ]:
penguins.select("species", "island", "year", "bill_length_mm").drop("year")

Not the most staggeringly complicated query, but we'll see more soon.

### Exercise 1

Your PI is a cranky American biologist who thinks the metric system is for
suckers (oh no).

He demands that we convert all of the existing measurements (`mm` and `g`) to
inches and lbs, respectively (I am so sorry). The PI is extra cranky this
morning, so we had better make sure that ONLY the silly units are visible in the
output.

Some ~helpful~ cursed conversion factors:

|  |  |
| -- | -- |
| mm -> in | divide by 25.4 |
| g -> lb | divide by 453.6 |

Two ways you might accomplish this:
- Chaining `.mutate` to create new columns and `.drop` to drop the old metric columns
- Using a single `.select` to create the new columns as well as select the relevant older columns

Try both ways below! How do they compare?

In [ ]:
# Convert the metric units to imperial, and drop the metric columns.
# Try this using a `.mutate` and `.drop` call.
penguins_imperial_mutate_drop = penguins

In [ ]:
# Convert the metric units to imperial, and drop the metric columns.
# Try this using a single `.select` call.
penguins_imperial_select = penguins

#### Solutions

In [ ]:
%load solutions/nb01_ex01_mutate_drop.py

In [ ]:
%load solutions/nb01_ex01_select.py

### Does it matter which method you choose?

In this case, no.  Sometimes, there might be a small difference in the generated
SQL but they will be semantically equivalent..

In [ ]:
ibis.to_sql(penguins_imperial_mutate_drop)

In [ ]:
ibis.to_sql(penguins_imperial_select)

In practice, small differences in the generated SQL don't make a difference.
Any modern SQL execution engine will optimize variations to the same set of
operations and there will be no measurable performance difference.

## Order By

Want to order your data by a given column or columns?  Use `order_by`!

The default ordering direction is ascending:

In [ ]:
penguins.order_by(penguins.flipper_length_mm)

We can ask Ibis to sort in descending order, too.

In [ ]:
penguins.order_by(penguins.flipper_length_mm.desc())

Let's select out a subset of the columns to keep this a bit tidier.

In [ ]:
penguins.order_by(penguins.flipper_length_mm.desc()).select(
    "species", "island", "flipper_length_mm"
)

You can also call `ibis.desc` on the column name to set the order direction:

In [ ]:
penguins.order_by(ibis.desc("flipper_length_mm")).select("species", "island", "flipper_length_mm")

## Aggregate

Ibis has several aggregate functions available to help summarize data.  All the
old favorites are there: `mean`, `max`, `min`, `count`, `sum`...

You can aggregate a column by calling the method on that column:

In [ ]:
penguins.flipper_length_mm.mean()

Or you can compute multiple aggregates using the `aggregate` method (also
available as `agg` for faster typing):

In [ ]:
penguins.agg([penguins.flipper_length_mm.mean(), penguins.bill_depth_mm.max()])

But aggregates really shine when paired with a `group_by`!

## Group By

`group_by` creates groupings of rows that have the same value for one or more columns.

But it doesn't do much on its own -- you can pair it with aggregate to get a result.

In [ ]:
penguins.group_by("species").agg()

Without any `aggregate` function specified, we get the distinct values of the grouped-column.

We can add a second column to the `group_by` to get the distinct pairs across both columns:

In [ ]:
penguins.group_by(["species", "island"]).agg()

Now, if we add an aggregation function to that, we start to really open things up.

In [ ]:
penguins.group_by(["species", "island"]).agg(penguins.bill_length_mm.mean())

By adding that `mean` to the `aggregate`, we now have a concise way to calculate
aggregates over each of the distinct groups in the `group_by`. And we can
calculate as many aggregates as we need.

In [ ]:
penguins.group_by(["species", "island"]).agg(
    [penguins.bill_length_mm.mean(), penguins.flipper_length_mm.max()]
)

If we need more specific groups, we can add to the `group_by`.

In [ ]:
penguins.group_by(["species", "island", "sex"]).agg(
        [penguins.bill_length_mm.mean(), penguins.flipper_length_mm.max()]
    )

## Exercises

Time to use what we've learned to answer some penguin questions.

### Exercise 2

What was the largest female penguin (by body mass) on each island in the year 2008?


#### Solution

Note that there are several ways these queries could be written - it's fine if
your solution doesn't look like ours, what's important is that the results are
the same.

In [ ]:
%load solutions/nb01_ex02.py

### Exercise 3

What was the largest male penguin (by body mass) on each island for each year of
data collection?


#### Solution

In [ ]:
%load solutions/nb01_ex03.py

## A brief digression on the SQL Ibis generates

Maybe you've heard that SQL has a standard?  This is true, and also misleading.
The SQL standard is more of a suggestion, and there are myriad SQL _dialects_.

Ibis compiles expressions into the appropriate SQL dialect for the backend you are using.
In this case, we started with a DuckDB table, so we get DuckDB SQL:

In [ ]:
ibis.to_sql(expr)

But if you want to use a _different_ dialect, you can pass the dialect name:

In [ ]:
ibis.to_sql(expr, dialect="postgres")

## Some useful extras

### `cast`

Sometimes when you parse data, _especially_ from CSVs, the types get a bit
messed up. Or you might be loading in a `parquet` file where everything is
defined as a `string`.  We can clean that up pretty quickly.

You can cast from floats to ints:

In [ ]:
penguins.bill_length_mm.cast("int32")

And from ints to floats:

In [ ]:
penguins.year.cast("float64")  # this is a terrible idea

You can cast numeric columns to strings:

In [ ]:
penguins.year.cast("str")  # or "string"

And numeric strings to numbers:

In [ ]:
penguins.year.cast("str").cast("int64")

But `ibis` will yell if you try to cast a non-numeric string to a number:

In [ ]:
penguins.species.cast("int32")

### dropna

Does what it says on the box -- drop the `NULL`s from a dataset.

In [ ]:
penguins.dropna()

1: Horst AM, Hill AP, Gorman KB (2020). palmerpenguins: Palmer Archipelago
(Antarctica) penguin data. R package version 0.1.0.
https://allisonhorst.github.io/palmerpenguins/. doi: 10.5281/zenodo.3960218.